# pySpark Data Validation

## Install pySpark

In [1]:
!pip install pyspark

### Import Spark Session

In [2]:
from pyspark.sql import SparkSession

#### Create spark object from the sparksession

In [3]:
spark = SparkSession.builder.appName("pySpark").getOrCreate()

#### Spark object details

In [4]:
spark

#### Read the input dataset for Data check

In [5]:
visitorsDF = spark.read\
.format('csv')\
.option('header',True)\
.option("TreatEmptyValuesAsNulls", True)\
.option("IgnoreLeadingWhiteSpace", True)\
.option("IgnoreTrailingWhiteSpace", True)\
.load('C:/Users/Manikandan Muthiah/mypython/data/data/VisitorLogsData.csv')

#### Import all necessary functions

In [6]:
from pyspark.sql.functions import *

#### 1.Number of columns in dataframe-Column count

In [7]:
columns=len(visitorsDF.columns)
columns

9

#### Checking null values in Spark dataframe(No of Empty columns)

In [8]:
nullcntdf=visitorsDF.select([count(when( col(c).isNull(),c)).alias(c) for c in visitorsDF.columns])
nullcntdf.show()

+-----------+-------------+---------+-------+--------+-------+---+-------+-------+
|webClientID|VisitDateTime|ProductID| UserID|Activity|Browser| OS|   City|Country|
+-----------+-------------+---------+-------+--------+-------+---+-------+-------+
|          0|       658915|   527137|5937305|  889446|      0|  0|2296200| 528062|
+-----------+-------------+---------+-------+--------+-------+---+-------+-------+



#### 3.Row Count of dataframe

In [10]:
total_rows=visitorsDF.count()
total_rows

6588000

#### Drop null values rows

In [11]:
visitorsDFnonull = visitorsDF.na.drop("any")

In [12]:
nonemptyrows=visitorsDFnonull.count()
nonemptyrows

284952

#### Completeness

In [13]:
completeness=(nonemptyrows/(total_rows)*100)
completeness

4.325318761384335

#### Datatype of dataframe

In [14]:
visitorsDF.printSchema()

root
 |-- webClientID: string (nullable = true)
 |-- VisitDateTime: string (nullable = true)
 |-- ProductID: string (nullable = true)
 |-- UserID: string (nullable = true)
 |-- Activity: string (nullable = true)
 |-- Browser: string (nullable = true)
 |-- OS: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)



In [15]:
nullcntdf.createOrReplaceTempView('nullcount')
columns_list=''
for c in visitorsDF.columns:
    columns_list=columns_list+c+'+'
columns_list=columns_list[:-1]

In [16]:
selectstmt="select sum({}) as totalnullcnt from nullcount".format(columns_list)

In [17]:
totalnullcnt=spark.sql(selectstmt).collect()
totalnullcnt=str(totalnullcnt)
totalnullcnt=totalnullcnt.split('=')[1].rsplit(')')[0]
totalnullcnt=int(totalnullcnt)

In [18]:
(totalnullcnt/(total_rows*columns))*100

18.277448896984417

#### Statistics of dataframe

In [19]:
visitorsDF.summary().show()

+-------+--------------+--------------------+---------+-------+--------+-------------+-------+-----------+---------------+
|summary|   webClientID|       VisitDateTime|ProductID| UserID|Activity|      Browser|     OS|       City|        Country|
+-------+--------------+--------------------+---------+-------+--------+-------------+-------+-----------+---------------+
|  count|       6588000|             5929085|  6060863| 650695| 5698554|      6588000|6588000|    4291800|        6059938|
|   mean|          null|1.526554171346754...|     null|   null|    null|         null|   null|       null|           null|
| stddev|          null|5.338116776808671E14|     null|   null|    null|         null|   null|       null|           null|
|    min|     WI1000000| 1525651211454000000| Pr100000|U100002|   CLICK|AdsBot-Google|Android|'Ain Benian|    Afghanistan|
|    25%|          null|1.526058871432999...|     null|   null|    null|         null|   null|       null|           null|
|    50%|       

#### Count of distinct values in Dataframe

In [20]:
visitorsDF.select([(countDistinct(col(c))).alias(c) for c in visitorsDF.columns]).show()

+-----------+-------------+---------+------+--------+-------+---+-----+-------+
|webClientID|VisitDateTime|ProductID|UserID|Activity|Browser| OS| City|Country|
+-----------+-------------+---------+------+--------+-------+---+-----+-------+
|    1091455|      5482669|    17459| 34050|       4|     82| 30|26259|  18912|
+-----------+-------------+---------+------+--------+-------+---+-----+-------+



#### Maximum length of each column in Dataframe

In [21]:
visitorsDF.select([max((length(col(c)))).alias(c) for c in visitorsDF.columns]).show()

+-----------+-------------+---------+------+--------+-------+---+----+-------+
|webClientID|VisitDateTime|ProductID|UserID|Activity|Browser| OS|City|Country|
+-----------+-------------+---------+------+--------+-------+---+----+-------+
|         15|           23|        8|     7|       8|     26| 13|  63|     70|
+-----------+-------------+---------+------+--------+-------+---+----+-------+



#### Minimum length of each column in Dataframe

In [22]:
visitorsDF.select([min((length(col(c)))).alias(c) for c in visitorsDF.columns]).show()

+-----------+-------------+---------+------+--------+-------+---+----+-------+
|webClientID|VisitDateTime|ProductID|UserID|Activity|Browser| OS|City|Country|
+-----------+-------------+---------+------+--------+-------+---+----+-------+
|          9|           19|        8|     7|       5|      2|  3|   2|      4|
+-----------+-------------+---------+------+--------+-------+---+----+-------+



#### Size of dataframe

In [23]:
size = (total_rows, columns)
print(size)

(6588000, 9)


#### Distinct rows count

In [24]:
visitorsDF.distinct().count()

6026927

#### Stop Spark

In [25]:
spark.stop()